In [ ]:
# Sannsynlighet

For det aller enkleste, som å ta union, snitt osv. mellom to sannsynligheter, gjør vi det enklest uten å hente inn eksterne pakker. Å skrive P(...) o.l. fungerer dårlig med Python sin syntaks, da dette ville blitt evaluert som en funksjon. Den peneste måten å få en skrivemåte som ligner på det vi er vandt til, er nok derfor å bruke dictionaries. Vi kan da lage en dict som heter P, og sette data inni denne, slik:

In [1]:
P = {} # Tom dict, vi legger inn data på de neste linjene.
P['A'] = 1/5 # Tilsvarer P(A)
P['B'] = 1/2 # osv...
P['B|A'] = 1/7

P # Vis hva som ligger inne i P-objektet vårt sålangt.

{'A': 0.2, 'B': 0.5, 'B|A': 0.14285714285714285}

Vi kan deretter gjøre utregninger ved å bruke reglene for kombinatorikk.

In [2]:
P['AB'] = P['B|A']*P['A']
print('P(AB) =', P['AB'])

P['AuB'] = P['A'] + P['B'] - P['AB']
print('P(AuB) =', P['AuB'])

P['A|B'] = (P['B|A']*P['A'])/P['B']
print('P(A|B) =', P['A|B'])

P(AB) = 0.02857142857142857
P(AuB) = 0.6714285714285714
P(A|B) = 0.05714285714285714


### Trekk fra en mengde

Det finnes dessverre ingen hurtigmetode i de pakkene vi har tilgjengelig, for å regne sannsynligheter ved trekk fra en mengde. Ferdig funksjonalitet får vi i større grad til senere kapitler. Vi må derfor implementere trekk-formlene selv, eller vi kan la datamaskinen gjøre jobben med å plukke verdier veldig mange ganger i en løkke. Dette kan man gjøre ved å bruke noe ala dette i en for-løkke:

In [3]:
import numpy as np
innhold = ['A', 'A', 'H', 'R', 'D', 'H', 'E']

# Henter fire tilfeldige verdier fra innhold, med tilbakelegging.
utvalg_tilbakelegg = np.random.choice(innhold, 4)
print('Med tilbakelegging:', utvalg_tilbakelegg)

# Henter fire tilfeldige verdier fra innhold, uten tilbakelegging.
utvalg_uten_tilbakelegg = np.random.choice(innhold, 4, replace=False)
print('Uten tilbakelegging:', utvalg_uten_tilbakelegg)

# Sjekke om vi fant H, E, R, D:
leter_etter = ['H','E','R','D']
fant_herd = (leter_etter == utvalg_uten_tilbakelegg).all() # Sjekker likhet mellom alle verdier i to lister.
print('Plukket H, E, R, D?', fant_herd)

# Hvis vi sorterer både verdiene vi leter etter, og listen vi plukket, spiller ikke rekkefølgen på utvalget
# lenger noen rolle.
leter_etter.sort() # leter_etter er nå [D, E, H, R], sortert alfabetisk
utvalg_uten_tilbakelegg.sort()
fant_herd = (leter_etter == utvalg_uten_tilbakelegg).all() # Sjekker likhet mellom alle verdier i to lister.
print('Plukket D, E, H, R?', fant_herd)

Med tilbakelegging: ['H' 'R' 'D' 'D']
Uten tilbakelegging: ['H' 'A' 'A' 'E']
Plukket H, E, R, D? False
Plukket D, E, H, R? False


Vi kan også, som nevnt, bygge våre egne funksjoner basert på trekk-formlene. Her er de fire trekk-formlene med to typer data. Man kan også bygge ut formlene for flere typer data ved bruk av multinomial-funksjonen vi lagde i forrige kapittel.

In [4]:
from scipy.misc import comb

def ordnet_tilbakelegg(p, n, k):
    return p**k * (1-p)**(n-k)

def ordnet_uten_tilbakelegg(N, n, S, k):
    return comb(N-n, S-k) / comb(N, S)

def uordnet_tilbakelegg(p, n, k):
    return comb(n, k) * ordnet_tilbakelegg(p, n, k)

def uordnet_uten_tilbakelegg(N, n, S, k):
    return comb(n, k) * ordnet_uten_tilbakelegg(N, n, S, k)

### Trekke drops på ulike måter

Vi skal nå trekke drops fra en bolle, på ulike måter. Først setter vi opp "drops", som er "bollen" vår. Vi teller så antall drops, og lager en ny dictionary med fargene som nøkkel, men sannsynlighet (antall/totalt antall) som verdi.

Deretter bruker vi formelen for tilbakelegging, og finner sannsynligheten for at vi trekker en gitt sekvens.

In [5]:
from collections import OrderedDict
drops = OrderedDict()
drops['O'] = 4
drops['R'] = 8
drops['G'] = 3
drops['B'] = 17
drops['L'] = 5

print('Drops:', drops)

# Henter alle tallverdiene og summerer antall drops:
antall_drops = sum(drops.values()) 

# Få en ny "dict" med farge som nøkkel og sannsynlighet for farge = antall/antall_drops
ps = OrderedDict()
for farge, antall in drops.items():
    ps[farge] =  antall/antall_drops
    
    
print('Sannsynlighet for farger:', ps)

print('---')

# GBBROBROL
# VIKTIG: I Python bruker vi ** for "opphøyd i", ikke ^. Så f.eks. er 2**3 lik 8.
farger_tilbakelegg = ps['O']**1 * ps['R']**2 * ps['G']**1 * ps['B']**4 * ps['L']**1
print('Sannsynligheten for å trekke BBOBRLBGR med tilbakelegging:', farger_tilbakelegg)

Drops: OrderedDict([('O', 4), ('R', 8), ('G', 3), ('B', 17), ('L', 5)])
Sannsynlighet for farger: OrderedDict([('O', 0.10810810810810811), ('R', 0.21621621621621623), ('G', 0.08108108108108109), ('B', 0.4594594594594595), ('L', 0.13513513513513514)])
---
Sannsynligheten for å trekke BBOBRLBGR med tilbakelegging: 2.467808144964132e-06


Videre kan vi lage en funksjon for sekvens uten tilbakelegging. Her må vi ha multinomialfunksjonen vi lagde i forrge kapittel. Vi lager også en liten hjelpefunksjon "multi_uten_tilbakelegg". Denne er veldig kompakt, og krever en del forståelse av Python for å forstå. Ikke fortvil om du ikke forstår koden. Det viktigste er at du forstår hvorfor vi bruker denne formelen i dette tilfellet. Hvis du kjører denne notepaden i Jupyter på din egen maskin, kan du bare bruke funksjonen slik den er, til å løse andre oppgaver!

In [6]:
# Fra forrige kapittel:
from scipy.misc import comb

def multinomial(n, ks):
    svar = comb(n, ks[0], exact=True) # Indekser i Python starter på 0. Binomial(n, k_1)
    
    # Trekker fra k-verdier fra n (k_1, k_2, ... k_n-2)
    # og bruker comb ved hver iterasjon for å multiplisere med tidligere svar.
    for m in range(1, len(ks)-1):
        n -= ks[m-1]
        svar *= comb(n, ks[m], exact=True)
        
    return svar

# Forklaring:
# N = Totalt antall drops
# n = Antall drops vi skal plukke
# Ss = Antall drops av hver farge
# ks = Antall drops vi skal plukke av hver farge
def multi_uten_tilbakelegg(N, n, Ss, ks):
    return multinomial(N-n, [Ss[i] - ks[i] for i in range(len(Ss))]) / multinomial(N, Ss)


farger_uten_tilbakelegg = multi_uten_tilbakelegg(
    N = antall_drops,
    n = 9,
    Ss = list(drops.values()), # Henter drops fra hver farge ut fra drops-dict, så vi slipper å skrive det på nytt.
    ks = [1, 2, 1, 4, 1]
)

print('Sannsynligheten for å trekke BBOBRLBGR uten tilbakelegging:', farger_uten_tilbakelegg)

Sannsynligheten for å trekke BBOBRLBGR uten tilbakelegging: 4.251394685210036e-06


### Gummibåtens dag

Vi har følgende scenario: Arne og vennene hans skal delta på gummibåtens dag. Dette er et arrangement hvor ungdommer setter seg i små gummibåter og farer ned elva Otra, fra Vennesla til Kristiansand. Arne har lite penger, og kjøper derfor en brukt gummibåt. Han kan velge mellom:

* Gummibåt A: Har fire deler som blåses opp hver for seg.
* Gummibåt B: Har seks deler som blåses opp hver for seg.

Begge gummibåtene vil synke hvis det går hull på halvparten av luftrommene. Det er 10% sjanse for at hvert rom ryker. Hvilken båt bør Arne kjøpe for å komme seg tryggest mulig gjennom strykene i elva?

Her trenger vi formelen for uordnet trekk med tilbakelegging, som vi definerte tidligere i kapittelet. Første parameter er sannsynligheten p, så totalt antall luftrom. Deretter kommer antall rom vi kan la sprekke, og som fremdeles vil få Arne og venne hans trygt frem.

In [16]:
gummibaat_a_synker = uordnet_tilbakelegg(0.1, 4, 2)
gummibaat_b_synker = uordnet_tilbakelegg(0.1, 6, 3)

print('Sjanse for at gummibåt A synker:', gummibaat_a_synker * 100, '%')
print('Sjanse for at gummibåt B synker:', gummibaat_b_synker * 100, '%')

Sjanse for at gummibåt A synker: 4.86 %
Sjanse for at gummibåt B synker: 1.458 %


Vi ser dermed at Arne bør kjøpe gummibåt B.

### Oppsummering

Akkurat under dette temaet får vi ikke så mye hjelp av Pandas, og de andre pakkene vi har brukt. Det fine med Python er at det er et fullverdig programmeringsspråk, så man kan selv skrive kode for funksjonalitet som ikke ligger ferdig tilgjengelig. Det kan virke vanskelig nå, men det åpnes mange dører, hvis man kan litt programmering. I de neste kapitlene får vi heldigvis mer hjelp til å løse oppgavene igjen!